## Our 4 Starting Question we decided to change to a single one:


1. How well can the level of corruption in a European country be quantified?

This questions is too broad and may lack specificity. Also not a good question to answer within a notebook.
The definition of "quantified" needs clarification—are we looking at an index, a model, or a metric?

2. Are there different forms of corruption prevalent in different European countries?

While interesting, it may be challenging to obtain granular, comparable data across countries.
Requires deep qualitative insights, which may not be fully captured by available datasets.

3. What characteristics of a country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

4. What characteristics of a country predict an increase or decrease in the level of corruption?

Why it's promising:

Focuses on change over time, enabling trend analysis.
Useful for policy evaluation and forecasting.
Encourages a deeper exploration of temporal datasets (e.g., economic reforms, governance improvements, etc.).
Can help identify early warning signs for rising corruption.
Potential Challenges:

Requires time-series data and careful handling of lag effects.
External factors (global economic crises, political events) may introduce noise.


The following question seems the most promising and we decided to only choose this one:

# What characteristics of a european country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

maybe we could tackle this one but lets find out later: Which country characteristics (e.g., economic, political, social indicators) best predict the level of corruption in European countries, and how do these characteristics relate to any gap between actual and perceived corruption?



# Start

As we have decided to focus on the continent of europe the first challenge will be to create a dataset of all european countries. To make this notebook not substantially long we will provide the finished data **europe_countries** to you. It shows all european countries and their ISO3 Code and ISO2 Code which should allow better preprocessing later.

In [1]:
import numpy as np
import pandas as pd

In [2]:
countries = pd.read_csv("../data/processed/europe_countries.csv")

In [24]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    48 non-null     object
 1   ISO3 Code  48 non-null     object
 2   ISO2 Code  48 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [25]:
countries['Country'].unique()

array(['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium',
       'Bosnia & Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
       'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
       'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom', 'Vatican City'], dtype=object)

When looking at european countrie numbers, many figures fly arround, our data includes 48 countries, and of course the most promising ones. As we will see later most of the other datasets aint serve data for all countries anyway. Especially for small countries like Vatican or Monaco, etc.

### Corruption Perceptions Index (CPI) from Transparency International.
Data Set that shows preceived corruption of countries and rank them.

Link to data: https://images.transparencycdn.org/images/CPI2023_FullDataSet.zip
https://www.transparency.org/en/news/how-cpi-scores-are-calculated

 0   Economy ISO3                                
 1   Economy Name                                 
 2   Year                                           
 3   Corruption Perceptions Index Rank            
 4   Corruption Perceptions Index Score           
 5   Corruption Perceptions Index Sources        
 6   Corruption Perceptions Index Standard Error

We took the xlsx and transformed it to this more comapct version with only features we need and only european countries. Data points get explained at the end of each processing.

In [5]:
cpi_data = pd.read_csv("../data/processed/CPI.csv")

In [13]:
iso3_europe_full = set(countries["ISO3 Code"])

iso3_europe_cpi = set(cpi_data["Economy ISO3"])

print('Lenght of all europe countries in our list: ',len(iso3_europe_full))
print('Lenght of cpi countriest:', len(iso3_europe_cpi))

print('Countries mssing')

iso3_europe_full-iso3_europe_cpi

Lenght of all europe countries in our list:  48
Lenght of cpi countriest: 42
Countries mssing


{'AND', 'LIE', 'MCO', 'RKS', 'SMR', 'VAT'}

Countries missing: Andora, Liechtenstein, Kosovo, San Marino, Vatikan, Gibraltar, Azerbaijan. We could neclect this countries due to their small size and impact.

In [21]:
cpi_data['Year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023])

In [23]:
cpi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Economy ISO3                                 504 non-null    object 
 1   Economy Name                                 504 non-null    object 
 2   Year                                         504 non-null    int64  
 3   Corruption Perceptions Index Rank            491 non-null    float64
 4   Corruption Perceptions Index Score           504 non-null    float64
 5   Corruption Perceptions Index Sources         504 non-null    float64
 6   Corruption Perceptions Index Standard Error  504 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 27.7+ KB


#### CPI Dataset Summary
* Corruption Perceptions Index Rank (float64):
This represents the ranking of the country based on its Corruption Perceptions Index (CPI) score, with rank 1 being the least corrupt country.
Lower ranks indicate better perceived transparency.

* Corruption Perceptions Index Score (float64):
This score quantifies perceived corruption levels on a scale of 0 to 100, where:
0 = highly corrupt
100 = very clean (low corruption perception)
It is calculated using multiple data sources, standardized, and aggregated.

* Corruption Perceptions Index Sources (float64):
The number of sources used to calculate the CPI score for a given country.
The CPI uses multiple expert assessments and business surveys; having more sources increases reliability.

*  Corruption Perceptions Index Standard Error (float64):
This represents the uncertainty or variability in the CPI score, showing how much variation exists among different data sources.
A lower standard error indicates a more reliable score, while a higher one suggests more disagreement or variability in corruption perceptions across sources.

### GDP Data

We decided to go for GDP per Capita (PPP - Purchasing Power Parity)

Why?

Adjusted for cost of living, making it a fairer comparison across countries.
Helps analyze the standard of living and economic well-being in relation to corruption.
Frequently used in corruption-related studies to measure economic development.

Definition: A country's gross domestic product (GDP) at purchasing power parity (PPP) per capita is the PPP value of all final goods and services produced within an economy in a given year, divided by the average (or mid-year) population for the same year. This is similar to nominal GDP per capita but adjusted for the cost of living in each country.



## Dataset Title

### 1. Basic Information
- **Data Source**: 
- **Description**: 
- **Year/Coverage**: 

### 2. Key Variables / Columns

| **Column Name** | **Type** | **Description** |
|-----------------|----------|-----------------|
| ...             | ...      | ...             |
| ...             | ...      | ...             |

### 3. Data Cleaning / Transformation
- **Original Format**: 
- **Filtering**: Note if you filtered the dataset (e.g., "Retained only European countries").
- **Columns Kept/Removed**: 
- **Data Type Conversions**: Mention any conversions (e.g., object → float).
- **Handling Missing Values**: 

### 4. Data Context & Usage
- **Why This Dataset**: 
- **Potential Biases**: 
- **Reliability Considerations**: 

### 5. Additional Notes / References
- **Official Documentation**: 
- **Version / Date of Retrieval**: 

In [50]:
gdp_ppp = pd.read_csv("../data/raw/gdp_per_capita.csv")

In [51]:

# gdp_ppp.drop(labels=["Series Code"], axis= 1)

#gdp_ppp = gdp_ppp.drop(gdp_ppp.tail(8).index)

gdp_ppp.columns = gdp_ppp.columns.str.replace(r"\[.*?\]", "", regex=True).str.strip()


gdp_ppp.head(10)


,Series Name,Series Code,Country Name,Country Code,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,GDP per capita (current US$),NY.GDP.PCAP.CD,Albania,ALB,4247.631343,4413.063383,4578.633208,3952.803574,4124.055390,4531.032207,5365.489347,5460.428237,5370.778623,6413.283286,6846.426143,8575.171134
1,GDP per capita (current US$),NY.GDP.PCAP.CD,Andorra,AND,41500.543579,42470.316116,44369.659691,38654.934720,40129.819201,40672.994335,42819.774580,41257.804585,37361.090067,42425.699676,42414.059009,46812.448449
2,GDP per capita (current US$),NY.GDP.PCAP.CD,Austria,AUT,48250.405914,50305.354577,51314.972262,43915.228021,45061.499392,47163.742578,51194.074984,49885.994736,48716.409890,53648.719074,52176.664914,56033.573792
3,GDP per capita (current US$),NY.GDP.PCAP.CD,Belarus,BLR,6953.215917,7998.080205,8341.290143,5967.068560,5039.775609,5785.533977,6360.053101,6837.768321,6542.864540,7489.718947,7994.648061,7829.053137
4,GDP per capita (current US$),NY.GDP.PCAP.CD,Belgium,BEL,44874.170918,46964.594678,47995.778696,40893.804538,41854.549830,44035.323936,47487.210039,46716.622747,45906.287581,51655.788330,50807.204708,54700.909324
5,GDP per capita (current US$),NY.GDP.PCAP.CD,Bosnia and Herzegovina,BIH,4693.788991,5034.566160,5206.065583,4662.258837,4928.566525,5345.019855,6047.845843,6122.375345,6130.311332,7295.343824,7655.594210,8638.666202
6,GDP per capita (current US$),NY.GDP.PCAP.CD,Bulgaria,BGR,7430.737380,7687.713682,7912.274844,7078.860323,7570.931655,8381.881346,9436.144799,9838.600590,10198.538831,12274.077595,13644.201583,15885.539131
7,GDP per capita (current US$),NY.GDP.PCAP.CD,Croatia,HRV,13508.683699,14136.396767,14187.969453,12285.303258,12821.173814,13902.631049,15460.418330,15563.816740,14808.462019,17789.930474,18466.095547,21865.456284
8,GDP per capita (current US$),NY.GDP.PCAP.CD,Cyprus,CYP,28991.935547,27797.457031,27244.351562,23487.228516,24715.154297,26696.990234,29599.986328,29703.052734,28649.015625,33734.292969,33938.945312,36551.417969
9,GDP per capita (current US$),NY.GDP.PCAP.CD,Czechia,CZE,20014.035402,20260.495500,20038.416318,17931.598767,18753.968672,20913.197632,23705.932981,24062.719001,23472.891545,27696.461953,28282.223672,31591.177073
